# Tutorial: candidate transiting exoplanets in Gaia DR3

## Author: Anthony Brown (Leiden Observatory)

In this notebook we will exercise the extraction of light curves of candidate exoplanet transits from the Gaia DR3 archive.

__To be finalized once the corrected vari_planetary_transits table is available from the Gaia web pages.__

## First install required Python packages

We need the `astroquery` package to access the tools for programmatic access to the Gaia archive. For more information and examples see:
* [astroquery documentation](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html)
* [Gaia archive python tutorial](https://www.cosmos.esa.int/web/gaia-users/archive/use-cases#ClusterAnalysisPythonTutorial)


## Python imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.gridspec import GridSpec

from astroquery.gaia import Gaia

from astropy.table import Table


## Matplotlib settings

In [ ]:
linewidths = 2
axislinewidths = 2
lenticks = 6
fontsize = 18

rc('font', family='sans-serif', size=fontsize)
rc('xtick.major', size=lenticks)
rc('xtick.minor', size=lenticks * 2 / 3)
rc('ytick.major', size=lenticks)
rc('ytick.minor', size=lenticks * 2 / 3)
rc('lines', linewidth=linewidths)
rc('axes', linewidth=axislinewidths)

## Function for plotting

The code below defines a function for plotting the light curves (epoch photometry) for the candidate exoplanet transits.

In [ ]:
def plot_epoch_phot(inp_table, colours  = ['C2', 'C0', 'C3'], title = 'Epoch photometry',  figsize = (15,6)):
    """
    Epoch photometry plotter. 'inp_table' MUST be an Astropy-table object.
    Code adapted from https://www.cosmos.esa.int/web/gaia-users/archive/Datalink-DR3
    
    Parameters
    ----------

    inp_table : Astropy table
        Table with lightcurve data

    Keywords
    --------

    colours: list of strings
        Matplotlib colours to use for the plotting
    title: string
        Plot title
    figsize: tuple of two floats
        Size of the figure
    """
    fig = plt.figure(figsize=figsize)
    gs = GridSpec(nrows=3, ncols=1, hspace=0, figure=fig)
    axes = []
    for i in range(3):
        axes.append(fig.add_subplot(gs[i,0]))
    xlabel   = f'JD date [{inp_table["time"].unit}]'
    ylabel   = f'magnitude [{inp_table["mag"].unit}]'
    gbands   = ['G', 'BP', 'RP']
    colours  = iter(colours)
    source_id = inp_table['source_id'][0]

    handles = []
    labels = []
    for band, ax in zip(gbands, axes):
        phot_set = inp_table[inp_table['band'] == band]
        valid = np.logical_not(np.isnan(phot_set['time']))
        ax.plot(phot_set['time'][valid], phot_set['mag'][valid], 'o', label = band, color = next(colours))
        ax.invert_yaxis()
        ax.grid()
        hs, ls = ax.get_legend_handles_labels()
        handles.append(hs[0])
        labels.append(ls[0])
    axes[2].set_xlabel(xlabel)
    axes[1].set_ylabel(ylabel)
    axes[0].legend(handles, labels, loc='upper left', bbox_to_anchor=(1, 0.95))
    fig.suptitle(title+f" Gaia DR3 {source_id}")
    plt.show()

# The Gaia DR3 planetary transits table
We first read and inspect the small table with information on the candidate exoplanet transits in Gaia DR3.

Download this file [VariPlanetaryTransit-INCORRECT.ecsv](https://catcopy.ipac.caltech.edu/ssw/hands-on/VariPlanetaryTransit-INCORRECT.ecsv), into the same directory as this notebook.

__To be updated once the corrected table is available from the Gaia web pages.__

In [ ]:
gtable = Table.read("VariPlanetaryTransit-INCORRECT.ecsv", format='ascii.ecsv')

In [ ]:
print(gtable.info)
gtable

We see a table with 214 entries. Detailed information on this table can be found [here](https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_variability_tables/ssec_dm_vari_planetary_transit.html).

## Getting the light curves
To access the associated light curves we use the *Datalink service*. The code below is based on the [tutorial](https://www.cosmos.esa.int/web/gaia-users/archive/datalink-products) in the Gaia archive.

In [ ]:
retrieval_type = 'EPOCH_PHOTOMETRY'
data_structure = 'INDIVIDUAL'
data_release   = 'Gaia DR3'

datalink  = Gaia.load_data(ids=gtable['source_id'], data_release = data_release, retrieval_type=retrieval_type, 
                           data_structure = data_structure, verbose = False, output_file = None, valid_data=False)
dl_keys  = [inp for inp in datalink.keys()]
dl_keys.sort()

print()
print(f'The following Datalink products have been downloaded (showing first five only):')
for dl_key in dl_keys[0:5]:
    print(f' * {dl_key}')

We now show the data available for the first confirmed exoplanet discovered by Gaia, Gaia-1b, which is Gaia DR3 3026325426682637824 (see [Panahi et al 2022](https://arxiv.org/abs/2205.10197)). We use the source ID to construct the key with which to extract the lightcurve data from the `datalink` object. The datalink entry can be turned into a regular Astropy table with the `to_table()` function call (`product.to_table()` below).

In [ ]:
source_id = 3026325426682637824 # Gaia 1-b
dl_key   = f'EPOCH_PHOTOMETRY-Gaia DR3 {source_id}.xml'

product  = datalink[dl_key][0]
items    = [item for item in product.iter_fields_and_params()]

for item in items:
    print(item)

print()
print(f'Showing data')
product

Note that the lightcurve data for all three of Gaia's broad photometric bands ($G$, $G_\mathrm{BP}$, $G_\mathrm{RP}$) is collected together in one table. You can select the band by filtering on the `band` column in the table (as done in the plotting code defined above). Next we plot an example lightcurve.

In [ ]:
plot_epoch_phot(product.to_table())

## __QUESTION 1__

The lightcurve as plotted above does not show the (candidate) exoplanet transit very well. This is better seen in a phase-folded lightcurve. 

Can you modify the plotting code above to plot phase-folded lightcurves?

In [ ]:
# Your modified code here

In [ ]:
# Code to make the plot

## __Challenge for the rest of the week__

1. Assuming the planet transit candidates are all indeed caused by exoplanets, calculate the radii of the planets, using the stellar radii from the Gaia archive. You can find the radii in the `gaiadr3.astrophysical_parameters` table.

2. Most of the candidate exoplanet transits correspond to known planets, see https://www.cosmos.esa.int/web/gaia/exoplanets. How do your radii compare to the literature values? You can use the literature values in the NASA Exoplanet Archive.

3. Given the planetary radii just calculated you can make an educated guess on their densities and calculate the masses. What does the mass distribution look like?